In [ ]:
# API installation
!pip install kagglehub

In [ ]:
!pip install sqlalchemy
!pip install psycopg2

In [ ]:
!pip install pandas

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as db 

In [2]:
import psycopg2
from psycopg2 import sql

In [3]:
from sqlalchemy import create_engine

## ETL PIPELINE

## EXTRACT

In [ ]:
# Import for kaggle API to retrieve data as CSV
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mrmars1010/banana-quality-dataset")

In [5]:
import os

In [6]:
# Utilizing OS module to find downloaded file
files = os.listdir(path)
print(files)

['banana_quality_dataset.csv']


In [ ]:
# Create file path
csv_file_path = os.path.join(path, files[0])  

In [8]:
df = pd.read_csv(csv_file_path)

In [9]:
df.head()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,Manzano,Colombia,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,Plantain,Guatemala,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,Burro,Ecuador,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,Manzano,Ecuador,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,Red Dacca,Ecuador,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [10]:
df.tail()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
995,996,Burro,Ecuador,3.50,Good,4.94,Ripe,21.06,2.49,27.95,196.35,2023-10-04,12.4,409.7,2029.9,173.0
996,997,Cavendish,Philippines,2.38,Processing,6.74,Overripe,16.10,2.00,12.39,165.55,2023-09-23,14.0,314.3,1330.7,180.5
997,998,Plantain,Ecuador,1.68,Processing,1.41,Green,17.17,2.28,16.56,210.08,2023-10-19,18.4,683.0,2955.4,189.9
998,999,Fehi,Guatemala,2.02,Processing,1.34,Green,17.03,2.88,26.31,162.50,2023-09-22,8.2,1362.7,1215.8,85.5
999,1000,Red Dacca,Ecuador,2.69,Good,2.69,Turning,21.10,1.23,21.95,105.39,2023-09-29,13.2,156.7,2863.9,51.3


In [11]:
# Displays overall info of the dataframe, including data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sample_id           1000 non-null   int64  
 1   variety             1000 non-null   object 
 2   region              1000 non-null   object 
 3   quality_score       1000 non-null   float64
 4   quality_category    1000 non-null   object 
 5   ripeness_index      1000 non-null   float64
 6   ripeness_category   1000 non-null   object 
 7   sugar_content_brix  1000 non-null   float64
 8   firmness_kgf        1000 non-null   float64
 9   length_cm           1000 non-null   float64
 10  weight_g            1000 non-null   float64
 11  harvest_date        1000 non-null   object 
 12  tree_age_years      1000 non-null   float64
 13  altitude_m          1000 non-null   float64
 14  rainfall_mm         1000 non-null   float64
 15  soil_nitrogen_ppm   1000 non-null   float64
dtypes: floa

In [12]:
df.shape

(1000, 16)

## TRANSFORM

In [13]:
# For checking nulls
print(df.isnull().sum())

sample_id             0
variety               0
region                0
quality_score         0
quality_category      0
ripeness_index        0
ripeness_category     0
sugar_content_brix    0
firmness_kgf          0
length_cm             0
weight_g              0
harvest_date          0
tree_age_years        0
altitude_m            0
rainfall_mm           0
soil_nitrogen_ppm     0
dtype: int64


In [14]:
# To remove duplicate entries
df.drop_duplicates(inplace=True)

In [15]:
df.shape

(1000, 16)

In [16]:
varieties = df[['variety']].drop_duplicates().sort_values(by='variety').reset_index(drop=True)

In [17]:
varieties = pd.DataFrame(varieties)

In [18]:
varieties['variety_id'] = varieties.index + 1

In [19]:
print(varieties.index)

RangeIndex(start=0, stop=8, step=1)


In [20]:
varieties

,variety,variety_id
0,Blue Java,1
1,Burro,2
2,Cavendish,3
3,Fehi,4
4,Lady Finger,5
5,Manzano,6
6,Plantain,7
7,Red Dacca,8


In [21]:
varieties = varieties[['variety_id', 'variety']]

In [22]:
varieties

,variety_id,variety
0,1,Blue Java
1,2,Burro
2,3,Cavendish
3,4,Fehi
4,5,Lady Finger
5,6,Manzano
6,7,Plantain
7,8,Red Dacca


In [23]:
regions = df[['region']].drop_duplicates().sort_values(by='region').reset_index(drop=True)

In [24]:
regions = pd.DataFrame(regions)

In [25]:
regions['region_id'] = regions.index + 1

In [26]:
print(regions.index)

RangeIndex(start=0, stop=8, step=1)


In [27]:
regions

,region,region_id
0,Brazil,1
1,Colombia,2
2,Costa Rica,3
3,Ecuador,4
4,Guatemala,5
5,Honduras,6
6,India,7
7,Philippines,8


In [28]:
regions = regions[['region_id', 'region']]

In [29]:
regions

,region_id,region
0,1,Brazil
1,2,Colombia
2,3,Costa Rica
3,4,Ecuador
4,5,Guatemala
5,6,Honduras
6,7,India
7,8,Philippines


In [30]:
mdf = pd.DataFrame(df)

In [31]:
mdf.head()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,Manzano,Colombia,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,Plantain,Guatemala,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,Burro,Ecuador,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,Manzano,Ecuador,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,Red Dacca,Ecuador,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [32]:
# Merge mdf with regions on the 'region' column
merged_df = pd.merge(mdf, regions, on='region', how='left')

In [33]:
# Replace 'region' in mdf with the 'Index' column from the merged dataframe
mdf['region'] = merged_df['region_id']

In [34]:
mdf.head()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,Manzano,2,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,Plantain,5,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,Burro,4,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,Manzano,4,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,Red Dacca,4,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [35]:
# Merge mdf with varieties on the 'variety' column
merged_df = pd.merge(mdf, varieties, on='variety', how='left')

In [36]:
# Replace 'variety' in mdf with the 'Index' column from the merged dataframe
mdf['variety'] = merged_df['variety_id']

In [37]:
mdf.head()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,6,2,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,7,5,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,2,4,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,6,4,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,8,4,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [38]:
df_m = pd.DataFrame(mdf)

In [39]:
df_m.head()

,sample_id,variety,region,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,6,2,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,7,5,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,2,4,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,6,4,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,8,4,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [40]:
# Rename columns to reflect id as reference
df_m = df_m.rename(columns={'variety': 'variety_id', 'region': 'region_id'})

In [41]:
df_m.head()

,sample_id,variety_id,region_id,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,6,2,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,7,5,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,2,4,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,6,4,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,8,4,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [42]:
regions_value = pd.DataFrame(regions[['region']])

In [43]:
regions_value

,region
0,Brazil
1,Colombia
2,Costa Rica
3,Ecuador
4,Guatemala
5,Honduras
6,India
7,Philippines


In [44]:
varieties_value = pd.DataFrame(varieties[['variety']])

In [45]:
varieties_value

,variety
0,Blue Java
1,Burro
2,Cavendish
3,Fehi
4,Lady Finger
5,Manzano
6,Plantain
7,Red Dacca


In [46]:
df_ms = pd.DataFrame(df_m)

In [47]:
df_ms['harvest_date'] = pd.to_datetime(df_ms['harvest_date'])

In [48]:
df_wt = pd.DataFrame(df_ms)

In [49]:
df_wt.head()

,sample_id,variety_id,region_id,quality_score,quality_category,ripeness_index,ripeness_category,sugar_content_brix,firmness_kgf,length_cm,weight_g,harvest_date,tree_age_years,altitude_m,rainfall_mm,soil_nitrogen_ppm
0,1,6,2,1.88,Processing,2.11,Turning,16.83,3.53,21.44,146.92,2023-10-16,13.7,58.2,2440.5,183.6
1,2,7,5,2.42,Processing,4.25,Ripe,16.73,4.09,26.11,160.48,2023-10-14,5.1,280.2,2374.6,109.8
2,3,2,4,3.57,Premium,6.24,Overripe,21.34,1.63,25.20,225.27,2023-09-08,17.7,1246.9,1191.5,147.7
3,4,6,4,2.21,Processing,5.39,Ripe,16.75,3.31,13.08,137.80,2023-10-07,13.0,1150.2,2845.1,92.8
4,5,8,4,2.35,Processing,5.84,Ripe,16.90,3.07,12.98,227.84,2023-10-02,4.8,526.0,2136.9,129.7


In [50]:
# Export DataFrame to CSV 
df_wt.to_csv('data.csv', index=False) 

## LOAD

## CREATE DATABASE

In [ ]:
user_password = '<password>'
user_port = '<port number>'

In [52]:
host = "localhost"  # Update with your database host
user = "postgres"   # Update with your PostgreSQL username
password = user_password  # Update with your PostgreSQL password
port = user_port  
new_database = "banana_quality"  # 'banana_quality'

# Connect to the PostgreSQL server
try:
    # Connect to the default 'postgres' database
    conn = psycopg2.connect(dbname="postgres", user=user, password=password, host=host, port=port)
    conn.autocommit = True  # Enable autocommit for database creation
    
    # Create a cursor
    cursor = conn.cursor()
    
    # Create the new database if it does not exist
    cursor.execute(sql.SQL("SELECT 1 FROM pg_database WHERE datname = %s"), [new_database])
    exists = cursor.fetchone()
    
    if not exists:
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(new_database)))
        print(f"Database '{new_database}' created successfully.")
    else:
        print(f"Database '{new_database}' already exists.")
    
    # Close the cursor and connection
    cursor.close()
    conn.close()

except Exception as e:
    print(f"Error: {e}")

Database 'banana_quality' created successfully.


## CREATE regions, varieties AND samples TABLES

In [53]:
# Database connection parameters

db_params = {
    'dbname': new_database,
    'user': 'postgres',
    'password': user_password,
    'host': 'localhost',  # or the IP of your PostgreSQL server
    'port': user_port,       # default PostgreSQL port
}

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    
    # SQL commands to create tables
    create_regions_table = """
    CREATE TABLE IF NOT EXISTS regions (
        region_id SERIAL PRIMARY KEY,
        region TEXT NOT NULL UNIQUE
    );
    """
    
    create_varieties_table = """
    CREATE TABLE IF NOT EXISTS varieties (
        variety_id SERIAL PRIMARY KEY,
        variety TEXT NOT NULL UNIQUE
    );
    """
    
    create_samples_table = """
    CREATE TABLE IF NOT EXISTS samples (
        sample_id SERIAL PRIMARY KEY,
        variety_id INTEGER NOT NULL,
        region_id INTEGER NOT NULL,
        quality_score DECIMAL(5,2),
        quality_category VARCHAR(50),
        ripeness_index DECIMAL(5,2),
        ripeness_category VARCHAR(50),
        sugar_content_brix DECIMAL(5,2),
        firmness_kgf DECIMAL(5,2),
        length_cm DECIMAL(5,2),
        weight_g DECIMAL(10,2),
        harvest_date DATE,
        tree_age_years DECIMAL(5,2),
        altitude_m DECIMAL(6,2),
        rainfall_mm DECIMAL(10,2),
        soil_nitrogen_ppm DECIMAL(10,2),
        FOREIGN KEY (variety_id) REFERENCES varieties (variety_id),
        FOREIGN KEY (region_id) REFERENCES regions (region_id)
    );
    """
    
    # Execute the SQL commands
    cursor.execute(create_regions_table)
    cursor.execute(create_varieties_table)
    cursor.execute(create_samples_table)
    
    # Commit changes to the database
    connection.commit()
    print("Tables 'regions', 'varieties', and 'samples' created successfully (or already exist).")
    
except (Exception, psycopg2.DatabaseError) as error:
    print("Error while creating PostgreSQL tables:", error)

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Tables 'regions', 'varieties', and 'samples' created successfully (or already exist).


## LOADING DATA

## INSERT INTO regions TABLE

In [54]:
try:
    # Connect to the PostgreSQL database
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            # Iterate over DataFrame rows
            for _, row in regions_value.iterrows():
                try:
                    # Insert each row into the regions table
                    cur.execute(
                        sql.SQL("INSERT INTO regions (region) VALUES (%s)"),
                        [row["region"]]
                    )
                except psycopg2.IntegrityError as e:
                    print(f"Error inserting region {row['region']}: {e}")
                    conn.rollback()  # Roll back in case of a UNIQUE constraint violation
                else:
                    conn.commit()  # Commit each successful insert

            print("Data inserted successfully!")

except psycopg2.Error as e:
    print(f"Database error: {e}")

Data inserted successfully!


## INSERT INTO varieties TABLE

In [55]:
try:
    # Connect to the PostgreSQL database
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            # Iterate over DataFrame rows
            for _, row in varieties_value.iterrows():
                try:
                    # Insert each row into the varieties table
                    cur.execute(
                        sql.SQL("INSERT INTO varieties (variety) VALUES (%s)"),
                        [row["variety"]]
                    )
                except psycopg2.IntegrityError as e:
                    print(f"Error inserting variety {row['variety']}: {e}")
                    conn.rollback()  # Roll back in case of a UNIQUE constraint violation
                else:
                    conn.commit()  # Commit each successful insert

            print("Data inserted successfully!")

except psycopg2.Error as e:
    print(f"Database error: {e}")

Data inserted successfully!


## INSERT INTO samples TABLE

In [56]:
# Database connection
conn = psycopg2.connect(dbname = new_database, user = "postgres", password = user_password, host = "localhost", port = user_port)
cur = conn.cursor()

try:
    # Step 1: Truncate the table to clear existing data
    cur.execute("TRUNCATE TABLE samples RESTART IDENTITY;")
    
    # Step 2: Load new data from the CSV file
    with open('data.csv', 'r') as f:
        next(f)  # Skip the header row
        cur.copy_expert("COPY samples FROM STDIN WITH CSV", f)

    # Commit the changes
    conn.commit()
    print("Data inserted successfully!")

except Exception as e:
    conn.rollback()
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    cur.close()
    conn.close()

Data inserted successfully!
